In [ ]:
# pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.7 MB/s eta 0:00:00


Below we use 7b param code llama 2 specialized for Python

In [ ]:
# import sentencepiece
# from transformers import LlamaForCausalLM, CodeLlamaTokenizer

# tokenizer = CodeLlamaTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")
# model = LlamaForCausalLM.from_pretrained("codellama/CodeLlama-7b-hf")

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# PROMPT = '''Write a function to find the length of the shortest string that has both str1 and str2 as subsequences.
# def super_seq(X, Y, m, n):
# <FILL_ME>
# return result
# '''
# input_ids = tokenizer(PROMPT, return_tensors="pt")["input_ids"]
# generated_ids = model.generate(input_ids, max_new_tokens=128)

# filling = tokenizer.batch_decode(generated_ids[:, input_ids.shape[1]:], skip_special_tokens = True)[0]
# print(PROMPT.replace("<FILL_ME>", filling))

In [1]:
def find_def_until_colon(input_string):
    def_index = input_string.find('def')

    if def_index != -1:
        colon_index = input_string.find(':', def_index)

        if colon_index != -1:
            return input_string[def_index:colon_index + 1]

    return None

In [2]:
def everything_after_def(input_string):
    def_index = input_string.find('def')

    if def_index != -1:
        double_newline_index = input_string.find('\n\n', def_index)
        if double_newline_index != -1:
            result = input_string[def_index:double_newline_index]
            return result
    return None

In [3]:
import re

def extract_args_with_function_name(declaration):
    pattern = r'def\s+(\w+)\s*\((.*?)\)'

    match = re.search(pattern, declaration)

    if match:
        function_name = match.group(1)
        arguments = match.group(2).split(', ')
    else:
        function_name = ""
        arguments = []

    if len(arguments) == 0:
        return function_name, "that takes no arguments"
    elif len(arguments) == 1:
        return function_name, f"that takes {arguments[0]} as an argument"
    else:
        args_except_last = ", ".join(arguments[:-1])
        last_arg = arguments[-1]
        return function_name, f"that takes {args_except_last} and {last_arg} as arguments"

In [4]:
def proper_format(prompt_in, code_in):
    str_1 = " "
    fun_name, str_2 = extract_args_with_function_name(code_in)
    function_index = prompt_in.find("to")
    if function_index != -1:
        result = prompt_in[function_index + len("to"):].strip()
    return result + " \n" + find_def_until_colon(code_in)

In [5]:
# def getCodeFormat(prompt_in, code_in):
#     prompt = proper_format(prompt_in, code_in)

#     input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]
#     attention_mask = torch.ones_like(input_ids)
#     pad_token_id = tokenizer.eos_token_id

#     generated_ids = model.generate(
#         input_ids,
#         attention_mask=attention_mask,
#         pad_token_id=pad_token_id,
#         max_new_tokens=128,
#         num_return_sequences=1
#     )

#     generated_code = tokenizer.batch_decode(
#         generated_ids[:, input_ids.shape[1]:],
#         skip_special_tokens=True
#     )[0]

#     # Assuming generated_code directly contains the code without placeholders
#     return generated_code


# print(getCodeFormat("Write a python function to find the length of the shortest word.", "def len_log(list1): min=len(list1[0]) for i in list1: if len(i)<min: min=len(i) return min"))

In [6]:
# def getCodeFormat(prompt_in, code_in):
#     PROMPT = proper_format(prompt_in, code_in)
#     input_ids = tokenizer(PROMPT, return_tensors="pt")["input_ids"]

#     attention_mask = torch.ones_like(input_ids)  # Set attention mask to ones (all tokens are attended to)
#     pad_token_id = tokenizer.eos_token_id  # Get the pad token ID

#     generated_ids = model.generate(input_ids, attention_mask=attention_mask, pad_token_id=pad_token_id, max_new_tokens=128, num_return_sequences=1)
#     filling = tokenizer.batch_decode(generated_ids[:, input_ids.shape[1]:], skip_special_tokens = True)[0]
#     print("hello" + filling)
#     responseText = PROMPT.replace("<FILL_ME>", filling)

#     return filling

# print(getCodeFormat("Write a python function to find the length of the shortest word.", "def len_log(list1): min=len(list1[0]) for i in list1: if len(i)<min: min=len(i) return min"))

In [7]:
!pip install git+https://github.com/huggingface/transformers.git@main accelerate

DEPRECATION: Loading egg at /Users/henrybae/miniconda3/envs/transformers/lib/python3.11/site-packages/huggingface_hub-0.19.4-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Cloning https://github.com/huggingface/transformers.git (to revision main) to /private/var/folders/0h/2pkv9xxd45lfrx3mx45bxhxw0000gn/T/pip-req-build-v0wsovfq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /private/var/folders/0h/2pkv9xxd45lfrx3mx45bxhxw0000gn/T/pip-req-build-v0wsovfq
  Resolved https://github.com/huggingface/transformers.git to commit fe1c16e95a1c2f7730b5d7340b669a63ab0b8ded
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 7.7 MB/s eta 0:00:00
  Att

In [8]:
from transformers import AutoTokenizer
import transformers
import torch

model = "codellama/CodeLlama-7b-Python-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

/Users/henrybae/miniconda3/envs/transformers/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
def getCodeFormat(prompt_input, code_input):
  sequences = pipeline(
      proper_format(prompt_input, code_input),
      do_sample=True,
      top_k=10,
      temperature=0.1,
      top_p=0.95,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_length=200,
  )
  return everything_after_def(sequences[0]['generated_text'])

# print(getCodeFormat("Write a python function to find the length of the shortest word.", "def len_log(list1): min=len(list1[0]) for i in list1: if len(i)<min: min=len(i) return min"))

In [ ]:
import locale
locale.getpreferredencoding = lambda: 'UTF-8'

In [ ]:
!pip install jsonlines

In [ ]:
import jsonlines
import torch


max_loop = 20
i = 0

with jsonlines.open('mbpp.jsonl') as reader:
# Loop through each example
    for obj in reader:
        prompt = obj['text']
        code = obj['code']

        print(find_def_until_colon(code))

        responseText = getCodeFormat(prompt, code)

        print(responseText)

        try:
          exec(responseText)
        except Exception as E:
          print("Code Didn't Run")
          print("Error" + str(E))
          continue

        # Assertion Code:
        assertion_code = obj['test_list']
        # Execute and print if the code passes the assertions
        for assertion in assertion_code:
            print(assertion)
            try:
                exec(assertion)
            except Exception as E:
                # Continue to the next example and print the error
                print("Code Failed Assertions" + str(E))
                break

        # terminate loop for testing
        i += 1
        if i == max_loop:
            break